## 特征提取

In [2]:
import psycopg2
from sklearn.feature_extraction import DictVectorizer

In [3]:
try:
    conn = psycopg2.connect("dbname='dpitest' user='cndmptest' host='60.205.147.49' password='Cptbtptp1401^'")
    #conn = psycopg2.connect("dbname='spiderdb' user='bupt_ant' host='10.105.32.54' password='123$%^qwe'")
except:
    print("cannot connect to the database!")

ip-category-pv

In [4]:
cur0 = conn.cursor()
"""select res.ip,res.category,sum(res.pv) from 
(select tag.ip,tag.pv,tag.rule_code,tag.series_code,s.series_name,s.category,rul.website_custom,tag.report_date
from gauss_car_dpiusertag tag 
left join gauss_car_mrrule rul on tag.rule_code=rul.code 
join gauss_car_series s on (s.series_code=tag.series_code and s.web_code=rul.website_custom)
where tag.ip in (select ip from gauss_car_dpiusertag group by ip having sum(pv)>100)) res 
group by res.ip,res.category order by res.ip,res.category"""
cur0.execute("""select res.ip,res.category,sum(res.pv) from
(select dpi.ip,dpi.pv,dpi.series_code,s.category,s.series_name
from dpiTop dpi
join gauss_car_mrrule rul on dpi.rule_code=rul.code 
join gauss_car_series s on (s.series_code=dpi.series_code and s.web_code=rul.website_custom)) res
group by res.ip,res.category
order by res.ip,res.category""")

In [5]:
ip_category = cur0.fetchall()

In [6]:
print(len(ip_category))
print(ip_category[:10])

59964
[('101.224.239.140', 'SUV', 39), ('101.224.239.140', '中型SUV ', 8), ('101.224.239.140', '中型车', 24), ('101.224.239.140', '中大型车', 4), ('101.224.239.140', '小型SUV ', 4), ('101.224.239.140', '小型车', 2), ('101.224.239.140', '紧凑型车', 54), ('101.224.239.140', '跑车', 2), ('101.224.239.140', None, 5), ('101.226.221.42', 'MPV ', 9)]


ip-province

In [7]:
cur1 = conn.cursor()

In [8]:
cur1.execute("select distinct ip,province from dpiTop order by ip")

In [9]:
ip_province = cur1.fetchall()

In [10]:
print(len(ip_province))
print(ip_province[:10])

5117
[('101.224.239.140', 'shanghai'), ('101.226.221.42', 'zhejiang'), ('101.228.81.112', 'shanghai'), ('101.228.89.24', 'shanghai'), ('101.228.99.190', 'shanghai'), ('101.229.1.71', 'shanghai'), ('101.229.251.173', 'shanghai'), ('101.230.193.54', 'shanghai'), ('101.230.213.210', 'shanghai'), ('101.231.138.254', 'shanghai')]


ip-series_code-pv

In [11]:
cur2 = conn.cursor()
cur2.execute("""select res.ip,res.series_name,sum(res.pv) from
(select dpi.ip,dpi.pv,dpi.series_code,s.category,s.series_name
from dpiTop dpi
join gauss_car_mrrule rul on dpi.rule_code=rul.code 
join gauss_car_series s on (s.series_code=dpi.series_code and s.web_code=rul.website_custom)) res
group by res.ip,res.series_name
order by res.ip,res.series_name""")

In [12]:
ip_series = cur2.fetchall()

In [13]:
print(len(ip_series))
print(ip_series[:10])

355535
[('101.224.239.140', 'GX6', 8), ('101.224.239.140', 'Yeti', 4), ('101.224.239.140', '丰田86', 2), ('101.224.239.140', '传祺GA8', 2), ('101.224.239.140', '卡罗拉', 2), ('101.224.239.140', '吉利博瑞', 17), ('101.224.239.140', '吉利博越', 5), ('101.224.239.140', '吉利帝豪EV', 1), ('101.224.239.140', '吉利帝豪GL', 3), ('101.224.239.140', '吉利新帝豪三厢', 1)]


ip-rule_code-pv

In [14]:
cur3 = conn.cursor()
cur3.execute("select ip,rule_code,sum(pv) from dpitop group by ip,rule_code order by ip,rule_code")

In [15]:
ip_ruleCode = cur3.fetchall()

In [16]:
print(len(ip_ruleCode))
print(ip_ruleCode[:10])

32682
[('101.224.239.140', '100_400', 2), ('101.224.239.140', '100_402', 2), ('101.224.239.140', '101_200', 44), ('101.224.239.140', '101_202', 68), ('101.224.239.140', '101_206', 62), ('101.224.239.140', '101_207', 1), ('101.224.239.140', '101_208', 2), ('101.224.239.140', '101_213', 2), ('101.224.239.140', '101_400', 2), ('101.224.239.140', '101_402', 3)]


all_ip

In [17]:
cur4 = conn.cursor()
cur4.execute("select distinct ip from dpitop order by ip")

In [18]:
all_ip = cur4.fetchall()

In [19]:
print(len(all_ip))
print(all_ip[:10])

4971
[('101.224.239.140',), ('101.226.221.42',), ('101.228.81.112',), ('101.228.89.24',), ('101.228.99.190',), ('101.229.1.71',), ('101.229.251.173',), ('101.230.193.54',), ('101.230.213.210',), ('101.231.138.254',)]


整合数据

In [20]:
all_data = {}
for item in all_ip:
    all_data[item[0]] = {}

In [21]:
print(all_data['101.224.239.140'])

{}


In [22]:
for pro in ip_province:
    all_data[pro[0]]['province'] = pro[-1]

In [23]:
print(all_data['101.224.239.140'])

{'province': 'shanghai'}


In [24]:
for cat in ip_category:
    if cat[1] is not None and cat[1] != 'null':
        all_data[cat[0]][cat[1].strip()] = 0

In [25]:
print(all_data['101.228.81.112'])

{'跑车': 0, '紧凑型SUV': 0, 'province': 'shanghai', '中大型车': 0, '中型车': 0, '大型SUV': 0, '中大型SUV': 0, '中型SUV': 0}


In [26]:
for cat in ip_category:
    if cat[1] is not None and cat[1] != 'null':
        all_data[cat[0]][cat[1].strip()] += cat[2]

In [27]:
print(all_data['101.228.81.112'])

{'跑车': 5, '紧凑型SUV': 25, 'province': 'shanghai', '中大型车': 45, '中型车': 1, '大型SUV': 10, '中大型SUV': 118, '中型SUV': 329}


In [28]:
for rul in ip_ruleCode:
    key = rul[1]
    if key is None:
        key = 'non-rule'
    all_data[rul[0]][key] = rul[2]

In [29]:
print(all_data['101.224.239.140'])

{'100_402': 2, '跑车': 2, '101_202': 68, '101_200': 44, '100_400': 2, '中大型车': 4, 'province': 'shanghai', '小型SUV': 4, '紧凑型车': 54, '101_208': 2, '101_402': 3, '101_400': 2, '101_206': 62, '101_207': 1, '小型车': 2, '中型车': 24, '中型SUV': 8, 'SUV': 39, '101_213': 2}


In [30]:
for ser in ip_series:
    key = ser[1]
    if key is not None and key != 'null':
        all_data[ser[0]][ser[1]] = ser[2]

In [31]:
print(all_data['101.224.239.140'])

{'100_402': 2, '吉利帝豪EV': 1, '跑车': 2, '中大型车': 4, '吉利新帝豪三厢': 1, '小型SUV': 4, 'Yeti': 4, '101_207': 1, '丰田86': 2, '荣威RX5': 29, '威朗VERANO': 4, '小型车': 2, '中型车': 24, '福克斯两厢': 1, '传祺GA8': 2, '威朗轿跑': 1, '101_208': 2, 'GX6': 8, '紧凑型车': 54, '骐达TIIDA': 4, '吉利博瑞': 17, '卡罗拉': 2, '吉利博越': 5, '翼搏': 2, '中型SUV': 8, '101_202': 68, 'SUV': 39, '君威': 1, '高尔夫': 2, '荣威350': 2, '100_400': 2, '新蒙迪欧': 2, '101_400': 2, '尚酷(进口)': 1, '君越': 2, '飞度': 2, '英朗': 2, '101_200': 44, '长安CS15': 4, '吉利帝豪GL': 3, '瑞风S3': 2, '朗逸': 3, '朗动': 4, '福克斯三厢': 1, 'province': 'shanghai', '思域': 1, '雪铁龙C3-XR': 2, '速腾': 8, '101_206': 62, '起亚K5': 4, '101_402': 3, '标致408': 13, '101_213': 2}


In [32]:
all_data_key = []
all_data_value = []
for key,val in all_data.items():
    all_data_key.append(key)
    all_data_value.append(val)

In [33]:
print(all_data_key[:10])
print(all_data_value[0])

['220.171.106.194', '115.182.24.99', '124.228.191.240', '117.94.120.114', '202.100.233.53', '139.205.148.240', '112.67.82.148', '60.169.119.74', '115.234.98.237', '120.68.43.6']
{'101_501': 1, '奥迪Q5': 1, '小型SUV': 35, '爱丽舍': 68, '宝沃BX7': 9, '撼路者': 10, '奇骏': 13, '中大型车': 5, '中型SUV': 39, '奔驰GLC': 16, '豪华车': 2, '绅宝D70': 1, '宝来两厢': 1, '普拉多': 14, '哈弗H1': 24, '轩逸': 73, '101_102': 37, 'Equinox': 3, '中型车': 17, '新轩逸': 16, '马自达CX-4': 2, '途观': 20, 'Equinox(海外)': 1, '哈弗H6': 14, '博越': 13, '传祺GA8': 8, '微型车': 5, '中大型SUV': 25, '传祺GS8': 40, '宝马4系': 1, '陆风X7': 13, '紧凑型车': 84, '锐界': 14, '野马T70': 12, '马自达CX-5': 2, '帝豪GS': 12, '雅特': 1, '101_202': 21, 'SUV': 19, '马自达2': 1, '熊猫': 17, '101_304': 2, '101_109': 10, '奔驰S级(进口)': 2, '101_100': 11, '101_503': 6, 'MODEL S': 1, '五菱宏光': 10, '阿特兹': 28, '观致3': 78, '锋驭': 13, '101_209': 1, '猎豹CS10': 33, '凯迪拉克ATS-L': 13, '比亚迪S7': 2, 'MPV': 4, '蓝鸟': 1, '逍客': 19, '101_103': 6, '蒂维拉': 22, '朗逸': 24, '帕萨特': 1, '101_200': 4, '紧凑型SUV': 55, '吉利SX7': 6, 'province': 'xinjiang', '速腾': 

In [34]:
print(all_data['180.174.187.23'])

{'跑车': 12, '普莱特': 1, '公爵(进口)': 2, 'Cross桑塔纳': 2, '雅力士': 1, '大众Eos(进口)': 10, '中大型车': 3, '101_204': 9, '北京现代ix25': 2, '小型车': 5, '中型车': 5, '欧迪玛(进口)': 1, '标致2008': 2, '考斯特(进口)': 1, '福克斯两厢': 12, '朗境': 4, '微型车': 1, '标致RCZ(进口)': 1, '紧凑型车': 85, '骐达TIIDA': 22, '翼搏': 4, '起亚KX3': 1, '格越(进口)': 3, '标致2008（进口）': 1, '高尔夫': 10, '101_202': 97, '林肯MKX(进口)': 14, 'SUV': 21, '陆地巡洋舰': 1, '高尔夫 嘉旅': 1, '宝驹': 1, 'K5（进口）': 1, '英朗': 3, '本田XR-V': 3, '帕萨特': 1, 'MPV': 2, '哥瑞': 3, 'Golf旅行轿车': 1, '朗逸': 5, '逍客': 2, '101_200': 37, 'province': 'shanghai', '嘉华': 1, '思域': 1, '朗行': 17, 'Teramont': 1, '101_206': 61, '客车': 1, '辉昂': 1, '福睿斯': 1, '起亚嘉华(进口)': 1, '丰田赛利卡(进口)': 1, '一汽-大众CC': 2, '桑塔纳·浩纳': 6}


In [35]:
#all_data_value[all_data_key.index('101.224.239.140')]
all_data_key.index('101.224.239.140')

4478

In [36]:
vec = DictVectorizer()

In [37]:
features = vec.fit_transform(all_data_value).toarray()

In [38]:
print(features.shape)

(4971, 4876)


In [39]:
print(features[1584])

[ 0.  0.  0. ...,  0.  0.  0.]


In [40]:
print(len(vec.get_feature_names()))
all_features = vec.get_feature_names()
print(all_features[:20])

4876
['01概念车', '100P', '100_300', '100_302', '100_303', '100_304', '100_305', '100_306', '100_307', '100_308', '100_400', '100_402', '100_403', '100_404', '100_405', '100_406', '100_407', '100_408', '100_410', '100_411']


In [41]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

##### normalize features

In [42]:
from sklearn.preprocessing import normalize

In [43]:
normalized_features = normalize(features, axis=0, norm='max')

In [44]:
normalized_features.shape

(4971, 4876)

## Clustering

In [45]:
from scipy.spatial.distance import cdist, pdist
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans,MiniBatchKMeans,DBSCAN,AgglomerativeClustering
import numpy as np

#### K-Means

In [46]:
K = range(1,15)
KM = [KMeans(n_clusters=k).fit(normalized_features) for k in K]

KeyboardInterrupt: 

In [ ]:
# Average silhouette method

s = []
for k_means in KM[1:]:
    
    labels = k_means.labels_
    centroids = k_means.cluster_centers_

    s.append(silhouette_score(normalized_features, labels, metric='euclidean'))

In [ ]:
print(s)

In [ ]:
plt.plot(range(2,15),s,'r')
plt.ylabel("Silouette")
plt.xlabel("k")
plt.title("Silouette for K-means cell's behaviour")
plt.show()

In [ ]:
for k_means in KM:
    labels = k_means.labels_
    print(np.bincount(labels))

In [ ]:
kmeans = KMeans(n_clusters=4).fit(normalized_features)

In [ ]:
labels_ = kmeans.labels_
print(labels_[:100])

In [ ]:
np.bincount(kmeans.labels_)

In [ ]:
labels_1 = np.where(labels_==3)[0]
labels_0 = np.where(labels_==0)[0]

In [ ]:
labels_0

In [ ]:
features.shape[1]

In [ ]:
cluster_1 = []
for i in labels_1:
    cluster_1.append(features[i])
print(type(cluster_1))

In [ ]:
cluster_0 = []
for i in labels_0:
    cluster_0.append(features[i])
print(cluster_0[:3])

In [ ]:
cluster_0 = np.array(cluster_0)

In [ ]:
cluster_1 = np.array(cluster_1)

In [ ]:
cluster_1_sum = np.sum(cluster_1, axis=0)

In [ ]:
print(cluster_1_sum)

In [ ]:
cluster_1_sum_big = cluster_1_sum.argsort()[-10:][::-1]  # 10 features whit biggest sum

In [ ]:
c1_f = []
c1_v =[]
for i in cluster_1_sum_big:
    print(all_features[i],cluster_1_sum[i])
    c1_f.append(all_features[i] if all_features[i] else 'None')
    c1_v.append(cluster_1_sum[i])

In [ ]:
cluster_0_sum = np.sum(cluster_0, axis=0)

In [ ]:
print(cluster_0_sum)

In [ ]:
cluster_0_sum_big = cluster_0_sum.argsort()[-10:][::-1]

In [ ]:
c0_f = []
c0_v = []
for i in cluster_0_sum_big:
    print(all_features[i],cluster_0_sum[i])
    c0_f.append(all_features[i] if all_features[i] else 'None')
    c0_v.append(cluster_0_sum[i])

In [ ]:
print(c0_f,c1_f)
c01_f = list(set(c1_f+c0_f))
print(c01_f)

In [ ]:
c0_vv = []
c1_vv = []
for f in c01_f:
    c0_vv.append(cluster_0_sum[all_features.index(f)])
    c1_vv.append(cluster_1_sum[all_features.index(f)])

In [ ]:
c0_vv = [i/np.bincount(kmeans.labels_)[0] for i in c0_vv]
c1_vv = [i/np.bincount(kmeans.labels_)[3] for i in c1_vv]

In [ ]:
print(c0_vv,c1_vv)

In [72]:
plt.plot(range(len(c01_f)),c0_vv,'r',range(len(c01_f)),c1_vv)
plt.xticks(range(len(c01_f)), c01_f)
plt.ylabel("PV")
plt.xlabel("Feature")
plt.show()

#### AgglomerativeClustering

In [ ]:
Aver = [AgglomerativeClustering(n_clusters=k, linkage='average',affinity='cosine').fit(normalized_features) for k in K]

In [ ]:
for aver in Aver:
    a_labels = aver.labels_
    print(np.bincount(a_labels))

In [ ]:
aver4 = Aver[4] 

In [ ]:
np.bincount(aver4.labels_)

#### DBSCAN

In [ ]:
db = DBSCAN().fit(normalized_features)

In [ ]:
labels = db.labels_

In [ ]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

In [ ]:
print(n_clusters_)

### Unsupervised dimensionality reduction

#### PCA

In [47]:
from sklearn.decomposition import PCA

In [48]:
features.shape

(4971, 4876)

In [49]:
features[:2]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  2.,  0.]])

In [51]:
pca = PCA().fit(features)

In [52]:
pca

PCA(copy=True, n_components=None, whiten=False)

In [53]:
eigenfaces = pca.components_.reshape((pca.n_components))

In [54]:
features_pca = pca.transform(features)

In [55]:
features_pca.shape

(4971, 4876)

In [56]:
features_pca[1]

array([ -2.85735172e+02,   9.47025192e+01,  -1.87179080e+01, ...,
         8.08692848e-16,  -7.72920953e-16,  -3.16451509e-17])

In [57]:
a_pca = AgglomerativeClustering(n_clusters=4, linkage='average',affinity='cosine').fit(features_pca)

In [58]:
print(np.bincount(a_pca.labels_))

[ 141 4786   43    1]


In [ ]:
Aver_pca = [AgglomerativeClustering(n_clusters=k, linkage='average',affinity='cosine').fit(features_pca) for k in K]

In [ ]:
for aver in Aver_pca:
    a_labels = aver.labels_
    print(np.bincount(a_labels))

#### TruncatedSVD  

In [59]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

In [60]:
svd = TruncatedSVD()
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

In [61]:
features_svd = lsa.fit_transform(features)

In [62]:
km = KMeans(n_clusters=3).fit(features_svd)

In [63]:
print(np.bincount(km.labels_))

[3688  896  387]


In [64]:
K = range(1,15)
KM_svd = [KMeans(n_clusters=k).fit(features_svd) for k in K]

In [65]:
for km in KM_svd:
    a_labels = km.labels_
    print(np.bincount(a_labels))

[4971]
[3804 1167]
[3688  387  896]
[ 711  837 3046  377]
[2727  834  371  409  630]
[2681  692  222  332  635  409]
[2626  348  237  406  541  175  638]
[1393  342  552  175  536  335 1406  232]
[1490  344  154  448  543  330  158 1281  223]
[1398  347  112  508  217  335  378 1360  154  162]
[ 219 1389  347  154  295  110  455  382  166  664  790]
[ 809  328  117  288  150  179  632  148  454  361  106 1399]
[1217  328  117  447  193  179  361  743  148  151  105  763  219]
[1142  328  111  140  230  147  361  709  785  414   90   85  179  250]


In [66]:
km_svd2 = KM_svd[1]
print(np.bincount(km_svd2.labels_))

[3804 1167]


In [67]:
labels_svd = km_svd2.labels_

In [68]:
l_1 = np.where(labels_svd==1)[0]
l_0 = np.where(labels_svd==0)[0]

In [69]:
cluster_svd_1 = []
for i in l_1:
    cluster_svd_1.append(features[i])
print(type(cluster_svd_1))

<class 'list'>


In [70]:
cluster_svd_0 = []
for i in l_0:
    cluster_svd_0.append(features[i])
print(type(cluster_svd_0))

<class 'list'>


In [71]:
cluster_svd_0 = np.array(cluster_svd_0)
cluster_svd_1 = np.array(cluster_svd_1)

In [72]:
cluster_svd_1_sum = np.sum(cluster_svd_1, axis=0)
cluster_svd_0_sum = np.sum(cluster_svd_0, axis=0)

In [73]:
cluster_svd_1_sum_big = cluster_svd_1_sum.argsort()[-10:][::-1]
cluster_svd_0_sum_big = cluster_svd_0_sum.argsort()[-10:][::-1]

In [74]:
c1_svd_f = []
c1_svd_v =[]
for i in cluster_svd_1_sum_big:
    print(all_features[i],cluster_svd_1_sum[i])
    c1_svd_f.append(all_features[i] if all_features[i] else 'None')
    c1_svd_v.append(cluster_svd_1_sum[i])

101_202 627024.0
101_200 510221.0
SUV 275477.0
紧凑型车 193727.0
中型车 103761.0
MPV 84258.0
101_206 80870.0
小型车 67923.0
皮卡 44166.0
跑车 43341.0


In [75]:
c0_svd_f = []
c0_svd_v =[]
for i in cluster_svd_0_sum_big:
    print(all_features[i],cluster_svd_0_sum[i])
    c0_svd_f.append(all_features[i] if all_features[i] else 'None')
    c0_svd_v.append(cluster_svd_0_sum[i])

紧凑型车 1385396.0
中型车 809618.0
紧凑型SUV 669614.0
101_402 641698.0
中型SUV 469116.0
101_103 451611.0
小型车 425482.0
小型SUV 397450.0
101_102 373877.0
MPV 306043.0


In [76]:
print(c0_svd_f,c1_svd_f)
c01_svd_f = list(set(c1_svd_f+c0_svd_f))
print(len(c01_svd_f),c01_svd_f)

['紧凑型车', '中型车', '紧凑型SUV', '101_402', '中型SUV', '101_103', '小型车', '小型SUV', '101_102', 'MPV'] ['101_202', '101_200', 'SUV', '紧凑型车', '中型车', 'MPV', '101_206', '小型车', '皮卡', '跑车']
16 ['中型SUV', '101_103', '跑车', '小型SUV', '101_200', '紧凑型SUV', '紧凑型车', '101_206', '皮卡', '101_102', '小型车', '101_202', '中型车', 'MPV', 'SUV', '101_402']


In [77]:
c0_svd_vv = []
c1_svd_vv = []
for f in c01_svd_f:
    c0_svd_vv.append(cluster_svd_0_sum[all_features.index(f)])
    c1_svd_vv.append(cluster_svd_1_sum[all_features.index(f)])

In [78]:
print(len(c0_svd_vv))

16


In [79]:
np.bincount(labels_svd)

array([3804, 1167])

In [80]:
c0_svd_vv = [i/(np.bincount(labels_svd)[0]) for i in c0_svd_vv]
c1_svd_vv = [i/(np.bincount(labels_svd)[1]) for i in c1_svd_vv]

In [81]:
print(len(c0_svd_vv),len(c1_svd_vv))

16 16


In [1]:
plt.plot(range(len(c01_svd_f)),c0_svd_vv,'r',range(len(c01_svd_f)),c1_svd_vv)
plt.xticks(range(len(c01_svd_f)), c01_svd_f)
plt.ylabel("PV")
plt.xlabel("Feature")
plt.show()

NameError: name 'plt' is not defined